# Analyzing Used Cars Listing Data on eBay

I will be working on analyzing the dataset containing used car listing info from _eBay_Kleinanzeigen_, a [claasifieds](https://en.wikipedia.org/wiki/Classified_advertising) section of the German eBay website.

The dataset was orginally uploaded on [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data) by user [orgesleka](https://www.kaggle.com/orgesleka). The original dataset is not available on Kaggle anymore but a copy can be found on [data.world](https://data.world/data-society/used-cars-data).

The version of the dataset we are working with is a sample of 50,000 data points that was prepared by Dataquest including simulating a less-cleaned version of the data.

Columns of the dataset contains information:
- `dateCrawled`: When this ad was first crawled. All field-values are taken from this date
- `Name`: The car's name
- `seller`: Private or dealer
- `offerType`: The type of listing
- `price`: The price of the used car
- `abtest`: Whether the listing is included in a A/B test
- `vehicleType`: The vehicle Type
- `yearOfRegistration`: The year in which which year the car was first registered
- `gearbox`: The transmission type
- `powerPS`: The power of the car in PS
- `model`: The car model name
- `odometer`: How many kilometers the car has driven
- `monthOfRegistration`: The month in which which year the car was first registered
- `fuelType`: What type of fuel the car uses
- `brand`: The brand of the car
- `notRepairedDamage`: If the car has a damage which is not yet repaired
- `dateCreated`: The date on which the eBay listing was created
- `nrOfPictures`: The number of pictures in the ad
- `postalCode`: The postal code for the location of the vehicle
- `lastSeen`: When the crawler saw this ad last online

Now let's start the project by constructing the dataframe.

In [1]:
import numpy as np
import pandas as pd

In [2]:
autos=pd.read_csv('autos.csv', encoding='Latin-1')
autos.index.name=None
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


There are 20 columns in the dataset and 16 of them are stored as string format. Null value exist in  columns including `vehicleType`, `gearbox`, `model`, `fuelType` and `notRepairedDamage`

We can make the following observations:
- The dataset contains 20 columns, most of which are strings.
- Some columns have null values, but none have more than ~20% null values.
- The column names use **camelcase** instead of Python's preferred **snakecase**, which means we can't just replace spaces with underscores.

We'll start by cleaning the column names to make our work easier.

## Clean Columns

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Let's transfer the column name into a snakecase and change a few wordings to name the columns properly.

In [4]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']

autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Exploring the Data and Initial Data Cleaning

In [5]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Findings:
- There are a few columns where nearly all of the values are the same, like `seller` and `offer_type`
- The `price` and `odometer` columns have numeric values stored in string format, we'll need to clean them for vectorized computing
- The `num_photos` columns looks odd, we need to investigate in this column later

In [6]:
autos['num_photos'].value_counts()

0    50000
Name: num_photos, dtype: int64

In [7]:
autos['seller'].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

In [8]:
autos['offer_type'].value_counts()

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64

We'll drop these 3 columns since they contribute little to our analysis

In [9]:
autos.drop(['num_photos', 'seller', 'offer_type'], axis=1)

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,"$24,900",control,limousine,2011,automatik,239,q5,"100,000km",1,diesel,audi,nein,2016-03-27 00:00:00,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,"$1,980",control,cabrio,1996,manuell,75,astra,"150,000km",5,benzin,opel,nein,2016-03-28 00:00:00,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,"$13,200",test,cabrio,2014,automatik,69,500,"5,000km",11,benzin,fiat,nein,2016-04-02 00:00:00,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,"$22,900",control,kombi,2013,manuell,150,a3,"40,000km",11,diesel,audi,nein,2016-03-08 00:00:00,35683,2016-04-05 16:45:07


Now let's convert the `price` and `odometer` columns.

In [10]:
autos['price']=autos['price'].str.replace('$','').str.replace(',','').astype(float)
autos['odometer']=autos['odometer'].str.replace('km','').str.replace(',','').astype(float)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8212\4085944508.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  autos['price']=autos['price'].str.replace('$','').str.replace(',','').astype(float)


In [11]:
autos['odometer'].head()

0    150000.0
1    150000.0
2     70000.0
3     70000.0
4    150000.0
Name: odometer, dtype: float64

In [12]:
autos['price'].head()

0    5000.0
1    8500.0
2    8990.0
3    4350.0
4    1350.0
Name: price, dtype: float64

## Exploring Odometer and Price

`price` column first

In [28]:
print(autos['price'].unique().shape)
print(autos['price'].value_counts().head(20))
print(autos['price'].describe())

(2357,)
0.0       1421
500.0      781
1500.0     734
2500.0     643
1000.0     639
1200.0     639
600.0      531
800.0      498
3500.0     498
2000.0     460
999.0      434
750.0      433
900.0      420
650.0      419
850.0      410
700.0      395
4500.0     394
300.0      384
2200.0     382
950.0      379
Name: price, dtype: int64
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


The price column seems well rounded, probably because the price takein system has pre-set options.

There are 1421 listings with 0 price, which accounted for 2.8% percent of the total listings, we maight consider deleting these rows directly. The hightse price is 100 million dollars, which requires more investigation into this.

In [32]:
print(autos['price'].value_counts().sort_index().head(20))

0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0       3
18.0       1
20.0       4
25.0       5
29.0       1
30.0       7
35.0       1
Name: price, dtype: int64


In [29]:
print(autos['price'].value_counts().sort_index(ascending=False).head(20))

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
259000.0      1
250000.0      1
220000.0      1
198000.0      1
197000.0      1
Name: price, dtype: int64


There are a number of listings with prices below \\$30, including about 1,500 at \\$0. There are also a small number of listings with very high values, including 14 at around or over \\$1 million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is \\$1. We will keep the \\$1 items, but remove anything above \\$350,000, since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [34]:
autos=autos[autos['price'].between(1,35000)]
print(autos['price'].describe())

count    48056.000000
mean      5331.427168
std       6047.229364
min          1.000000
25%       1200.000000
50%       2999.000000
75%       7200.000000
max      35000.000000
Name: price, dtype: float64


Next let's explore `odometer` column

In [36]:
print(autos['odometer'].describe())
print(autos['odometer'].value_counts().sort_index().head(20))

count     48056.000000
mean     126488.263692
std       39140.930914
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer, dtype: float64
5000.0        803
10000.0       221
20000.0       693
30000.0       725
40000.0       766
50000.0       963
60000.0      1112
70000.0      1185
80000.0      1390
90000.0      1710
100000.0     2089
125000.0     5025
150000.0    31374
Name: odometer, dtype: int64


There are more high mileage than low mileage vehicles and the numbers are also rounded.

## Exploring the Date Columns

Columns containing date information are:
- `date_crawled`
- `registration_year`
- `registration_month`
- `ad_created`
- `last_seen`

Here we'll explore these columns and make some modifications to them

In [44]:
print(autos[['date_crawled', 'ad_created', 'last_seen']][:5])
print(autos[['date_crawled', 'ad_created', 'last_seen']][:5].info())

          date_crawled           ad_created            last_seen
0  2016-03-26 17:47:46  2016-03-26 00:00:00  2016-04-06 06:45:54
1  2016-04-04 13:38:56  2016-04-04 00:00:00  2016-04-06 14:45:08
2  2016-03-26 18:57:24  2016-03-26 00:00:00  2016-04-06 20:15:37
3  2016-03-12 16:58:10  2016-03-12 00:00:00  2016-03-15 03:16:28
4  2016-04-01 14:38:50  2016-04-01 00:00:00  2016-04-01 14:38:50
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date_crawled  5 non-null      object
 1   ad_created    5 non-null      object
 2   last_seen     5 non-null      object
dtypes: object(3)
memory usage: 160.0+ bytes
None


The date information from these 3 columns are stored as strings and we'll inspect them one by one. 

In [48]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025366
2016-03-06    0.014067
2016-03-07    0.036083
2016-03-08    0.033378
2016-03-09    0.033149
2016-03-10    0.032233
2016-03-11    0.032483
2016-03-12    0.037082
2016-03-13    0.015690
2016-03-14    0.036603
2016-03-15    0.034293
2016-03-16    0.029653
2016-03-17    0.031630
2016-03-18    0.012860
2016-03-19    0.034689
2016-03-20    0.037872
2016-03-21    0.037144
2016-03-22    0.032920
2016-03-23    0.032233
2016-03-24    0.029278
2016-03-25    0.031526
2016-03-26    0.032379
2016-03-27    0.031047
2016-03-28    0.034793
2016-03-29    0.034189
2016-03-30    0.033752
2016-03-31    0.031775
2016-04-01    0.033607
2016-04-02    0.035438
2016-04-03    0.038684
2016-04-04    0.036582
2016-04-05    0.012922
2016-04-06    0.003184
2016-04-07    0.001415
Name: date_crawled, dtype: float64

Seems like the date was crawled daily in a month from March to April and the distribution of crawled listings are roughly uniform.

In [49]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001082
2016-03-06    0.004370
2016-03-07    0.005390
2016-03-08    0.007491
2016-03-09    0.009655
2016-03-10    0.010758
2016-03-11    0.012465
2016-03-12    0.023930
2016-03-13    0.008948
2016-03-14    0.012631
2016-03-15    0.015981
2016-03-16    0.016460
2016-03-17    0.028196
2016-03-18    0.007346
2016-03-19    0.015919
2016-03-20    0.020643
2016-03-21    0.020663
2016-03-22    0.021475
2016-03-23    0.018603
2016-03-24    0.019810
2016-03-25    0.019269
2016-03-26    0.016814
2016-03-27    0.015690
2016-03-28    0.020996
2016-03-29    0.022474
2016-03-30    0.024908
2016-03-31    0.023826
2016-04-01    0.022911
2016-04-02    0.025012
2016-04-03    0.025283
2016-04-04    0.024659
2016-04-05    0.124001
2016-04-06    0.221346
2016-04-07    0.130993
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [55]:
print(autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index())
print(autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_values(ascending=False).head(50))

2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
                ...   
2016-04-03    0.038913
2016-04-04    0.036978
2016-04-05    0.011632
2016-04-06    0.003267
2016-04-07    0.001269
Name: ad_created, Length: 75, dtype: float64
2016-04-03    0.038913
2016-03-20    0.037935
2016-03-21    0.037311
2016-04-04    0.036978
2016-03-12    0.036915
2016-03-14    0.035209
2016-04-02    0.035105
2016-03-28    0.034938
2016-03-07    0.034793
2016-03-29    0.034106
2016-03-15    0.034064
2016-03-19    0.033607
2016-04-01    0.033607
2016-03-30    0.033565
2016-03-08    0.033378
2016-03-09    0.033232
2016-03-11    0.032837
2016-03-22    0.032774
2016-03-26    0.032421
2016-03-23    0.032088
2016-03-10    0.031921
2016-03-31    0.031838
2016-03-25    0.031671
2016-03-17    0.031234
2016-03-27    0.030943
2016-03-16    0.030173
2016-03-24    0.029216
2016-03-05    0.022911
2016-03-13    0.017022
2016-03-06    0.015357
2016-03-18  

The `ad_created` column indicated that the ad was created in a large time period, with the oldest ad created on 2015-08-10. The distribution of the time, however, illustrates that most ad_created time fall within 1-2 months of the listing time.

In [56]:
autos['registration_year'].describe()

count    48056.000000
mean      2004.635862
std         87.023293
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

`registration_year` column has a minimum value of 1000 and a maximum value of 9999, both of which are incorrect given there is no time machine, we'll correct them later.

## Modifying the Incorrect Registration Year Data

Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's see the proportion of invalid data and then decide whether we need to delete them.

In [61]:
print(1-autos.loc[autos['registration_year'].between(1900,2016),'registration_year'].shape[0]/autos.shape[0])

0.039058598301980996


Invalid data only existed in about 4% percents of the entire dataset, therefore it's safe for us to drop them. 

In [67]:
autos=autos[autos['registration_year'].between(1900,2016)]
autos['registration_year'].value_counts(normalize=True, dropna=False).sort_values(ascending=False).head(20)

2000    0.068299
2005    0.063492
1999    0.062712
2003    0.058425
2004    0.058338
2006    0.057732
2001    0.057039
2002    0.053747
1998    0.051084
2007    0.048918
2008    0.047489
2009    0.044912
1997    0.042162
2011    0.034345
2010    0.033825
1996    0.029689
2012    0.026787
1995    0.026462
2016    0.026116
2013    0.016111
Name: registration_year, dtype: float64

It seems that most cars are registrated in the past 20 years.

## Analyzing Price by Brand

In [70]:
print(autos['brand'].value_counts(normalize=True, dropna=False))
print(len(autos['brand'].unique()))

volkswagen        0.212867
bmw               0.109769
opel              0.108729
mercedes_benz     0.095498
audi              0.085299
ford              0.070443
renault           0.047641
peugeot           0.030165
fiat              0.025921
seat              0.018472
skoda             0.016588
nissan            0.015440
mazda             0.015332
smart             0.014314
citroen           0.014162
toyota            0.012820
hyundai           0.010134
sonstige_autos    0.009247
volvo             0.009247
mini              0.008857
mitsubishi        0.008294
honda             0.007904
kia               0.007146
alfa_romeo        0.006691
suzuki            0.005998
chevrolet         0.005717
chrysler          0.003551
porsche           0.003378
dacia             0.002664
daihatsu          0.002534
subaru            0.002165
jeep              0.002144
land_rover        0.001754
saab              0.001667
daewoo            0.001516
jaguar            0.001516
trabant           0.001408
r

German manufacturers represent four out of the top five brands, almost 50% of the overall listings. Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.

There are lots of brands that don't have a significant percentage of listings, so we will limit our analysis to brands representing more than 5% of total listings.

In [72]:
brand_counts=autos['brand'].value_counts(normalize=True)
common_brands=brand_counts[brand_counts>0.05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [76]:
mean_price={}

for item in common_brands:
    mean=round(autos.loc[autos['brand']==item,'price'].sum()/autos[autos['brand']==item].shape[0])
    mean_price[item]=mean

print(mean_price)

{'volkswagen': 5280, 'bmw': 7742, 'opel': 2968, 'mercedes_benz': 7727, 'audi': 8378, 'ford': 3577}


Of the top 6 brands, there is a distinct price gap:
- Audi, BMW and Mercedes Benz are more expensive
- Ford and Opel are less expensive
- Volkswagen is in between - this may explain its popularity, it may be a 'best of 'both worlds' option.

## Analyzing the Mileage

In [77]:
mp_series=pd.Series(mean_price)
pd.DataFrame(mp_series, columns=['mean_price'])

,mean_price
volkswagen,5280
bmw,7742
opel,2968
mercedes_benz,7727
audi,8378
ford,3577


In [84]:
mean_mileage = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mile = brand_only["odometer"].mean()
    mean_mileage[brand] = int(mean_mile)

mm_series = pd.Series(mean_mileage).sort_values(ascending=False)
brand_info = pd.DataFrame(mm_series,columns=['mean_mileage'])
brand_info

,mean_mileage
bmw,133689
mercedes_benz,132235
audi,131315
opel,129310
volkswagen,128961
ford,124495


In [86]:
brand_info['mean_prices']=mp_series
brand_info

,mean_mileage,mean_prices
bmw,133689,7742
mercedes_benz,132235,7727
audi,131315,8378
opel,129310,2968
volkswagen,128961,5280
ford,124495,3577


The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.